# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 10. Prompt Cache 工程化实现方法与使用技巧</center>

&emsp;&emsp;`Prompt Cache` (提示缓存) 是`API`生成响应的临时存储机制。当客户端多次发送相同的提示（`Prompt`）时到大模型服务时（比如`DeepSeek`的`API`服务），可以重复使用缓存中的内容，从而用来减少响应延迟和计算开销。

&emsp;&emsp;通常场景下，一个模型接收到最终有效的`Prompt`一般是`system` 消息 + `user` 消息。即如下形式：

```python
    messages: [
        {"role": "system", "content": "你是一位乐于助人的助手"},
        {"role": "user", "content": "你好，我是木羽，很高兴认识你"}
    ]
```

&emsp;&emsp;经常存在的情况是：`system` 消息是固定的，而`user` 消息是变化的。除此以外，当涉及到多轮对话的时候，为了能让大模型具备历史的上下文信息，往往需要将之前所有的对话历史都拼接起来，作为新的`Prompt` 进行传递。即如下形式：

```python
    # 第一轮对话
    messages: [
        {"role": "system", "content": "你是一位乐于助人的助手"},
        {"role": "user", "content": "你好，我是木羽，很高兴认识你"}
    ]
    # 第二轮对话    
    messages: [
        {"role": "system", "content": "你是一位乐于助人的助手"},
        {"role": "user", "content": "你好，我是木羽，很高兴认识你"}
        {"role": "assistant", "content": "你好，木羽。"}
        {"role": "user", "content": "请问我叫什么名字？"}
    ]
```
    

&emsp;&emsp;`Prompt Cache` 做的事情就是存储并利用这些`Prompt` 中的重复内容，来降低响应延迟和推理成本。

# 1. DeepSeek 硬盘上下文缓存

&emsp;&emsp;从应用开发的角度看，大模型服务响应一个比较关键的概念就是`首 Token 延迟`，即从客户端发送请求到模型返回第一个`Token` 的时间间隔。比如我们正在开发的`AssistGen`应用全部采用流式输出接口，`首 Token 延迟`的直观感受就是用户看到第一个问题答案的时间。所以为了提升用户体验，需要尽可能的去降低`首 Token 延迟`的响应时间。像模型的服务商如`OpenAI`、`Claude`、`Gemini` 等都推出了`Prompt Cache` 接口特性。同样，`DeepSeek` 的在线`API`服务也支持使用`Prompt Cache` （提示缓存），并且无需修改代码，无需更换接口，硬盘缓存服务会自动运行，并自动按照实际命中情况计费。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251918343.png" width=80%></div>

&emsp;&emsp;`Prompt Cache` 能很好的解决`首 Token 延迟`的问题。 大模型在接收到输入的时候，需要先处理完整个输入才能开始生成输出，输入的序列越长，自然开始生成输出的时间就越长，所以通过 `Prompt cache`，大模型可以直接利用已缓存的处理结果，显著减少这个首次等待时间。

&emsp;&emsp;如下图所示，在多轮对话中，为了能让大模型具备历史的上下文信息，往往需要将之前所有的对话历史都拼接起来，作为新的`Prompt` 进行传递。因此，对于第2轮和第3轮的对话，`Prompt Cache` 的作用是能够重复使用前面1轮的响应，而不是从头开始生成新的响应，从而加快响应时间，并最大程度地减少计算工作量。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261038390.png" width=60%></div>

&emsp;&emsp;这里我们可以通过调用`DeepSeek` 的`API` 接口来验证`Prompt Cache` 的效果。这里需要注意的是：`DeepSeek` 的 `/chat/completions`是一个“无状态” `API`，即服务端不记录请求的上下文，所以在每次请求时，需将之前所有对话历史拼接好后进行传递。其中，在返回的响应结构中，`usage` 字段中会包含`prompt_cache_hit_tokens` 和`prompt_cache_miss_tokens` 字段，分别表示缓存命中和未命中的`Token` 数量。代码如下：

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

# 加载 .env 文件
load_dotenv()

# 实例化 DeepSeek API 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'), 
    base_url=os.getenv('DEEPSEEK_BASE_URL')
)


def chat_with_cache(messages):
    """单轮对话，返回响应并打印缓存情况"""
    start_time = time.time()
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    
    elapsed_time = time.time() - start_time
    
    # 打印缓存命中情况
    cache_hit = response.usage.prompt_cache_hit_tokens
    cache_miss = response.usage.prompt_cache_miss_tokens
    print(f"\n[耗时 {elapsed_time:.2f}s]")
    print(f"缓存命中: {cache_hit} tokens")
    print(f"缓存未命中: {cache_miss} tokens")
    
    return response.choices[0].message

def main():
    # 初始化对话历史
    messages = [
        {"role": "system", "content": "你是一位乐于助人的助手"}
    ]
    
    print("开始对话 (输入 'q' 退出):")
    
    try:
        while True:
            # 获取用户输入
            user_input = input("\n用户: ")
            if user_input.lower() == 'q':
                break
            
            # 添加用户消息
            messages.append({"role": "user", "content": user_input})
            
            # 获取模型回复
            assistant_message = chat_with_cache(messages)
            
            # 添加模型消息到历史列表中
            messages.append({
                "role": assistant_message.role,
                "content": assistant_message.content
            })
            
            # 打印模型回复
            print(f"AI助手: {assistant_message.content}")
            
    except KeyboardInterrupt:
        print("\n对话已终止")
    except Exception as e:
        print(f"\n发生错误: {e}")

if __name__ == "__main__":
    main()


开始对话 (输入 'q' 退出):


&emsp;&emsp;大家也可以进入`llm_backend\app\test\deepseek_disk_cache.py` 文件，运行代码，验证`Prompt Cache` 的效果。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261103544.png" width=80%></div>

&emsp;&emsp;通过上图可以看到，第一轮对话中，缓存命中数为0，表示没有命中缓存，需要从头开始生成新的响应。第二轮对话中，缓存命中数为128。`DeepSeek` 的缓存命中规则是：<font color=red>只有当两个请求的前缀内容相同时（从第 0 个 token 开始相同），才算重复。中间开始的重复不能被缓存命中。</font>， 因此第二轮对话中，缓存命中的内容就是第一轮对话的输入 + 第一轮对话的输出。


&emsp;&emsp;同时，`DeepSeek` 的缓存时间有效时间是几个小时到几天（官方文档说明），超出这个时间不再使用后缓存会被自动清空。


&emsp;&emsp;以上就是`Deepseek` 官方 `API` 提供的硬盘缓存，大家需要重点了解。在实际使用的时候，对于一些系统指令、背景信息或者常用的工具定义等内容，放在提示词的开头位置，而将动态的内容放在结尾，这种应用方式就能极大程度的利用到缓存的优势。

# 2. 使用Redis实现服务请求缓存

&emsp;&emsp;正如上一小结内容所提，`Prompt Cache` 和 大模型推理优化技术`KV-Cache` 本质上都是利用“复用之前的计算结果”的方式来提高响应速度，其更偏向于模型架构、服务架构层。如果大家想更进一步的了解其实现原理，可以详细阅读`OpenAI`发布的论文：[Prompt Cache：Modular Attention Reuse for Low-Latency Inference](https://arxiv.org/pdf/2311.04934)，这里不做扩展讲解。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261252588.png" width=100%></div>

&emsp;&emsp;提示缓存除了在模型层进行优化外，根据平台或者特定用例也可以用不同的方式实现。在这里，我们重点介绍一个后端服务常用的缓存解决方案：<font color='red'>**使用数据库缓存大模型对话的数据，当命中缓存后，则不向`API`服务发起调用，直接通过缓存提取某个问题的答案。**</font>

&emsp;&emsp;简单来说，当用户将问题（Prompt）发送到后端服务时，首先检查缓存中是否有直接可用的响应。如果检索到，则直接返回缓存中的响应。否则，再向`API`发送新的请求，再结合`API`服务的缓存机制得到最终的响应。

&emsp;&emsp;缓存的概念一直是计算中一种基本优化技术。在其最基本的形式中，缓存涉及将经常访问的数据存储在与原始数据源相比更快的位置中。比如：
- 硬件缓存：`CPU`使用多个级别的缓存（L1，L2，L3）存储经常访问的说明和数据，从而减少了从较慢的主内存中获取信息所需的时间。
- Web缓存： `Web`浏览器和服务器使用缓存来存储静态内容，减少经常访问的网页的负载时间。
- 数据库缓存：数据库系统采用缓存机制来存储查询结果或经常访问的数据，从而改善重复查询的响应时间。

&emsp;&emsp;在这种场景需求下，主流且最常用的还是内存数据库`Redis`和`Memcached`，其中`Memcached`相较于 `Redis`更加轻量，而`Redis`则能提供更加丰富的功能，同时具备比较高性能的读写操作，适合需要快速访问的场景。因此本节，我们就借助 `Redis` 来实现后端服务的`Prompt Cache`。

## 2.1 安装 Redis

&emsp;&emsp;`Redis` 是一个高性能且开源的内存数据库，它提供了丰富的数据结构和功能，适合用于缓存和存储。大家可以在[Redis官网](https://redis.io/)了解`Redis`的更多信息，同时也能在[Redis Github](https://github.com/redis/redis)获取`Redis`的源码。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261328657.png" width=80%></div>

&emsp;&emsp;`Redis` 支持多种操作系统，包括 `Linux`、`macOS`、`Windows` 等，但是[Redis官网](https://redis.io/docs/latest/operate/oss_and_stack/install/install-redis/)提供的安装教程有非常多的坑，并且描述过于简洁，导致大部分人都无法按照官方的操作文档顺利安装。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261331978.png" width=80%></div>


&emsp;&emsp;因此本节，我们就分别以 `Windows` 和 `Linux` 为例，来通过源码安装 `Redis`，尽可能的降低安装难度同时也更好的适配大家的开发环境。

&emsp;&emsp;首先，我们来看 `Windows` 系统的安装。

### 2.1.1 Window源码安装Redis

&emsp;&emsp;`Redis` 官方没有提供 `Windows` 版本的二进制包， 在`Windows`系统上运行官方推荐的方法是安装`WSL`后，使用`Linux`的子系统来运行`Redis`。如果大家的电脑已经安装了`WSL`，那么可以直接根据官方的[安装教程](https://redis.io/docs/latest/operate/oss_and_stack/install/install-redis/install-redis-on-windows/)进行安装。而如果不想因为一个`Redis`而安装`WSL`，毕竟一个完整的操作系统还是占用挺多系统资源的，则可以直接使用源码编译的方式来安装`Redis`。具体的操作方法如下：

- Step 1. 下载Redis的Windows的编译版本


&emsp;&emsp;`Github`上有很多`Redis`的`Windows`编译版本的项目，这里推荐的一个项目地址为：地址为[redis-windows](https://github.com/redis-windows/redis-windows)，其最高编译的`Redis`版本为`7.4.2`，和`Redis`官方最新版本保持一致。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261512062.png" width=80%></div>

&emsp;&emsp;点击`Releases`按钮，即可看到`Redis`的`Windows`所有编译版本。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261512063.png" width=80%></div>

&emsp;&emsp;不建议大家选择最新的版本，因为一般最新的版本可能存在一些问题，这里我们选择`7.0.8`版本，可以点击如下链接直接进行下载：https://github.com/redis-windows/redis-windows/releases/tag/7.0.8


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261515807.png" width=80%></div>

&emsp;&emsp;下载完成后，解压文件，得到`redis-7.0.8-win64-x64-msys2`文件夹。我这里存储到了`D:\redis-7.0.8-win64-x64-msys2`目录下。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261522957.png" width=80%></div>

- Step 2. 启动Redis服务

&emsp;&emsp;进入`Windows`系统的命令行终端（`cmd`），在命令行终端先进入`D:\redis-7.0.8-win64-x64-msys2`目录下，在执行如下命令启动 `redis` 服务：

```bash
    cd d:\redis-7.0.8-win64-x64-msys2   # 这里替换为自己的解压路径
    redis-server redis.conf
```



<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261517628.png" width=80%></div>

&emsp;&emsp;执行命令后，可以在命令行终端看到`Redis`服务启动的日志信息。


- Step 3. 连接Redis服务测试


&emsp;&emsp;启动`Redis`服务后，后端服务需要通过`Python`的`redis`库来连接`Redis`服务进行对其相关的操作。其调用示例在官方有较为详细的文档：https://redis.readthedocs.io/en/stable/examples.html， 我们这里先进行`Redis`的连接测试。


&emsp;&emsp;这里需要关注的关键连接信息是：`Redis`的默认连接地址是`127.0.0.1`，端口号是`6379`，数据库是`0`。在 `Redis` 中，数据库 0 是默认的数据库。`Redis` 默认提供了 16 个逻辑数据库，编号从 0 到 15。每个数据库都是独立的，具有自己的键空间。

In [ ]:
# pip install redis  # 安装redis库

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

&emsp;&emsp;建立连接后，我们就可以对`Redis`进行操作了。 相关的增删改查示例都非常简单，代码如下：


In [ ]:
# 增：添加数据
def add_data(key, value):
    r.set(key, value)           # set：设置键值对
    print(f"添加数据: {key} -> {value}")

# 查：获取数据
def get_data(key):
    value = r.get(key)         # get：获取键值对
    if value is not None:
        print(f"获取数据: {key} -> {value.decode('utf-8')}")
    else:
        print(f"键 {key} 不存在")

# 改：更新数据
def update_data(key, new_value):
    if r.exists(key):           # exists：判断键是否存在
        r.set(key, new_value)   # set：设置键值对，如果键存在，则更新键值对
        print(f"更新数据: {key} -> {new_value}")
    else:
        print(f"键 {key} 不存在，无法更新")

# 删：删除数据
def delete_data(key):
    if r.delete(key):    # delete：删除键值对
        print(f"删除数据: {key}")
    else:
        print(f"键 {key} 不存在，无法删除")


&emsp;&emsp;可以调用`add_data`、`get_data`、`update_data`、`delete_data`函数来对`Redis`进行操作。

In [ ]:
# 添加数据
add_data("name", "muyu")
add_data("age", "30")

添加数据: name -> muyu
添加数据: age -> 30


In [ ]:
# 查数据
get_data("name")
get_data("age")

获取数据: name -> muyu
获取数据: age -> 30


In [ ]:
# 改数据
update_data("age", "31")

# 查更新后的数据
get_data("age")


更新数据: age -> 31
获取数据: age -> 31


In [ ]:
# 删数据
delete_data("name")

# 查已删除的数据
get_data("name")

删除数据: name
键 name 不存在


&emsp;&emsp;注意：如果大家在启动的过程中，遇到类似 `MSYS2` 的错误，则需要大家先安装 `MSYS2`，然后再安装 `Redis`，具体的安装方法如下所示：（如果可以直接正常启动，则可以跳过该步骤）

---

&emsp;&emsp;MSYS2 是一个在 Windows 上模拟 Linux 环境的工具，它提供了一个类似于 Linux 的命令行界面，并且可以安装和运行许多 Linux 软件。其官网为[MSYS2](https://www.msys2.org/)，我们直接点击`Download the installer`官网的下载链接，下载安装包。<font color=red>注意：操作系统的版本要求是 64位Windows 10或更新。</font>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402086.png" width=80%></div>

&emsp;&emsp;下载完成后，我们双击安装包，按照提示进行安装。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402088.png" width=80%></div>

&emsp;&emsp;这里可以选择安装的路径，默认是`C:\msys64`，大家根据实际情况选择即可。我这里选择的是`D:\msys64`。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402089.png" width=80%></div>

&emsp;&emsp;接下来选择默认的选项，点击`Next`。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402090.png" width=80%></div>

&emsp;&emsp;这一步的安装过程比较漫长，大家耐心等待。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402091.png" width=80%></div>

&emsp;&emsp;等待安装后，我们可以在`D:\msys64`目录下找到`ucrt64.exe`文件，双击运行，即可打开`MSYS2`的命令行。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261409206.png" width=80%></div>

&emsp;&emsp;命令行界面如下所示，我们后续所有的操作都是在该命令行中进行。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261402092.png" width=80%></div>

### 2.2.2 Linux源码安装Redis

&emsp;&emsp;`Linux`系统上安装`Redis`，成功率最高的安装方式同样是源码编译安装的方式。具体的操作方法如下：

- Step 1. 下载`Redis`源码文件

&emsp;&emsp;首先通过`XShell`或者`FinalShell`连接到`Linux`服务器，安装编译打包需要的工具。命令如下：

```bash
    yum install -y gcc make tcl wget  # linux系统安装gcc、make、tcl、wget工具

    apt-get install -y gcc make tcl wget  # ubuntu系统安装gcc、make、tcl、wget工具
```

&emsp;&emsp;我的操作系统是`Ubuntu`，所以使用的是`apt-get`命令。如果大家的操作系统是`CentOS`，则使用`yum`命令。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261613197.png" width=80%></div>

- Step 2. 下载`Redis`源码文件

&emsp;&emsp;`Redis`的所有发行版本官方的下载地址为[Redis Releases](https://download.redis.io/releases/)，大家可以任意选择版本，但是不建议选择最新的版本，因为最新的版本可能存在一些问题。这里我们选择使用的版本是`7.0.4`。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261617106.png" width=80%></div>

&emsp;&emsp;在命令行界面。使用`wget`命令下载`Redis-7.0.4.tar.gz`的源码文件。命令如下：

```bash
    pwd  # 查看当前目录
    wget http://download.redis.io/releases/redis-7.0.4.tar.gz  # 下载redis源码，注意下载的位置为当前目录
``` 


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261613198.png" width=80%></div>

&emsp;&emsp;如果大家的服务器没有外网的话，可以先下载到本地，然后上传到服务器。

- Step 3. 解压`Redis`源码文件

&emsp;&emsp;下载完成后，我们可以在当前目录下看到`redis-7.0.4.tar.gz`文件。使用`tar`命令解压文件。命令如下：

```bash
    tar -zxvf redis-7.0.4.tar.gz
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261620833.png" width=80%></div>

- Step 4. 编译安装`Redis`

&emsp;&emsp;解压完成后，我们可以在当前目录下看到`redis-7.0.4`文件夹。使用`cd`命令进入`Redis`源码目录。命令如下：

```bash
    cd redis-7.0.4
```

&emsp;&emsp;构建`Redis`依赖库, 执行如下命令：（如果大家有多个`CPU`，可以添加`-j`参数，比如`make -j4`，表示使用4个`CPU`进行编译。）

```bash
    cd deps; make -j4 hiredis jemalloc linenoise lua
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261626573.png" width=80%></div>

&emsp;&emsp;依次执行：

```bash
    cd ..   # 回到redis源码目录
    make install  # 安装redis
    make clean  # 清理编译文件
``` 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261636888.png" width=80%></div>

- Step 5. 启动`Redis`服务


&emsp;&emsp;安装完成后，通过`redis-server`命令指定`Redis`的配置文件`redis.conf`，启动`Redis`服务。命令如下：

```bash
    cd src   # redis-server文件所在目录
    redis-server ../redis.conf  # 指定redis.conf文件
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261649105.png" width=80%></div>

&emsp;&emsp;至此，在`Linux`系统上，`Redis`的源码安装就完成了。并且成功启动了`Redis`服务。

- Step 6. 连接`Redis`服务测试


&emsp;&emsp;启动`Redis`服务后，后端服务需要通过`Python`的`redis`库来连接`Redis`服务进行对其相关的操作。其调用示例在官方有较为详细的文档：https://redis.readthedocs.io/en/stable/examples.html， 我们这里先进行`Redis`的连接测试。

&emsp;&emsp;这里需要关注的关键连接信息是：`Redis`的默认连接地址是`127.0.0.1`，端口号是`6379`，数据库是`0`。在 `Redis` 中，数据库 0 是默认的数据库。`Redis` 默认提供了 16 个逻辑数据库，编号从 0 到 15。每个数据库都是独立的，具有自己的键空间。

In [ ]:
# pip install redis  # 安装redis库

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

&emsp;&emsp;建立连接后，我们就可以对`Redis`进行操作了。 相关的增删改查示例都非常简单，代码如下：


In [ ]:
# 增：添加数据
def add_data(key, value):
    r.set(key, value)           # set：设置键值对
    print(f"添加数据: {key} -> {value}")

# 查：获取数据
def get_data(key):
    value = r.get(key)         # get：获取键值对
    if value is not None:
        print(f"获取数据: {key} -> {value.decode('utf-8')}")
    else:
        print(f"键 {key} 不存在")

# 改：更新数据
def update_data(key, new_value):
    if r.exists(key):           # exists：判断键是否存在
        r.set(key, new_value)   # set：设置键值对，如果键存在，则更新键值对
        print(f"更新数据: {key} -> {new_value}")
    else:
        print(f"键 {key} 不存在，无法更新")

# 删：删除数据
def delete_data(key):
    if r.delete(key):    # delete：删除键值对
        print(f"删除数据: {key}")
    else:
        print(f"键 {key} 不存在，无法删除")


&emsp;&emsp;可以调用`add_data`、`get_data`、`update_data`、`delete_data`函数来对`Redis`进行操作。

In [ ]:
# 添加数据
add_data("name", "muyu")
add_data("age", "30")

添加数据: name -> muyu
添加数据: age -> 30


In [ ]:
# 查数据
get_data("name")
get_data("age")

获取数据: name -> muyu
获取数据: age -> 30


In [ ]:
# 改数据
update_data("age", "31")

# 查更新后的数据
get_data("age")


更新数据: age -> 31
获取数据: age -> 31


In [ ]:
# 删数据
delete_data("name")

# 查已删除的数据
get_data("name")

删除数据: name
键 name 不存在


### 2.2.3 Redis配置项修改

&emsp;&emsp;`Redis`的配置文件为`redis.conf`，大家可以在`Redis`的源码目录下找到该文件。该配置文件中的内容非常多，这里我们介绍如下几个重要的配置项：

1. `bind`：指定`Redis`的监听地址，默认是`127.0.0.1`，如果需要监听所有地址，则可以设置为`0.0.0.0`。比如大家使用云服务器部署`Redis`，想要在自己的`IP`地址上进行访问，则可以设置为`0.0.0.0`，否则会无法访问到`Redis`服务。

2. `port`：指定`Redis`的监听端口号，默认是`6379`。如果大家想要修改端口号，则可以设置为其他端口号。

3. `requirepass`：指定`Redis`的密码，默认是`""`，如果大家想要设置密码，则可以设置为其他密码。

4. `protected-mode`：指定`Redis`的保护模式，默认是`yes`，如果大家想要关闭保护模式，则可以设置为`no`。

&emsp;&emsp;修改的方法也非常简单，大家只需要在`redis.conf`文件中找到对应的配置项，然后进行修改即可。比如在`Linux`系统上，可以通过`vim`命令修改`redis.conf`文件。命令如下：

```bash
    vim redis.conf
``` 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261711703.png" width=100%></div>

&emsp;&emsp;找到对应的配置项，然后进行修改，然后保存退出。重新启动`Redis`服务，即可生效。

## 2.2 Redis 在模型调用中的缓存应用

&emsp;&emsp;在部署并连接了`Redis`服务后，我们就可以在模型调用中使用`Redis`的缓存机制了。

&emsp;&emsp;现在我们就来实现这个需求：当用户将问题（Prompt）发送到后端服务时，先检查缓存中是否可用的响应。如果检索到了，直接返回缓存中的响应。否则，再向`API`发送新的请求，得到最终的响应。

&emsp;&emsp;这个机制对于需要重复或者常用的查询或者问题很有效，具体的执行处理逻辑如下：

1. 首先将用户输入进来的`Prompt`生成一个唯一的哈希键，作为缓存键；
2. 使用这个缓存键在`Redis`中查找是否存在对应的缓存响应；（即某个`Prompt`是否已经调用过模型，并且将响应结果存储到了缓存中）
3. 如果有缓存命中率（找到响应），它将立即返回缓存的响应，不再去重新执行大模型服务调用；
4. 如果未找到缓存错过（不存在响应），则将`Prompt`提示发送到大模型生成响应；
5. 将生成的响应存储到`Redis`中，以便后续的请求可以快速返回；
6. 返回生成的响应给用户。

&emsp;&emsp;这里首先使用`DeepSeek`的`DeepSeek-Chat`模型来实现基于`Redis`的缓存机制。现在来看一下具体的代码实现。

&emsp;&emsp;首先，先连接`Redis`服务，代码如下：

In [11]:
import redis

# 连接到 Redis
try:
    r = redis.Redis(
        host='192.168.110.131',   # 这里的`host`和`port`需要根据实际情况进行修改
        password='snowball2019',
        port=6379, 
        db=0)  

    # 测试连接
    r.ping()  # 发送 PING 命令以测试连接
    print("成功连接到 Redis！")
except redis.ConnectionError:
    print("无法连接到 Redis！请检查 Redis 服务是否正在运行。")
except Exception as e:
    print(f"发生错误: {e}")

成功连接到 Redis！


&emsp;&emsp;如果看到`成功连接到 Redis！`，则表示`Redis`服务连接成功。否则需要检查`Redis`服务是否正常运行，以及`host`和`port`是否正确。

- 调用`DeepSeek-Chat`模型，并对输入的`Prompt`进行哈希处理，生成唯一的哈希键。

&emsp;&emsp;哈希的作用是：将输入的`Prompt`转换为一个唯一的哈希值。针对不同长度的`Prompt`，其生成的哈希值是等长的。<font color="red">虽然哈希值本身并不包含原始文本的信息，但它可以唯一标识特定的输入，避免重复存储相同的内容。</font>。

In [1]:
import hashlib

question_1 = "你好，我是木羽，很高兴认识你"

# 因为哈希函数需要处理字节数据，而不是字符串，所以需要通过 encode 转换为字节串（bytes）
md5_hash = hashlib.md5(question_1.encode('utf-8')).hexdigest()  # 返回的是一个32位的十六进制字符串
print(md5_hash)

question_2 = "哈哈，你好"
md5_hash = hashlib.md5(question_2.encode('utf-8')).hexdigest()
print(md5_hash)

f6c96a1c978407d928bd19e7f13ee18f
e522ecc43cab6ba1e2abf2eb11b59691


&emsp;&emsp;哈希值的长度是固定的，不管输入的`Prompt`有多长，其生成的哈希值的长度都是固定的。同时，同样的`Prompt`，其生成的哈希值是唯一的。

In [2]:
import hashlib

question_1 = "你好，我是木羽，很高兴认识你"
md5_hash = hashlib.md5(question_1.encode('utf-8')).hexdigest()
print(md5_hash)

f6c96a1c978407d928bd19e7f13ee18f


&emsp;&emsp;了解了基本原理，我们实际调用`DeepSeek-Chat`模型，并对输入的`Prompt`进行哈希处理，生成唯一的哈希键。代码如下：

In [8]:
import hashlib
import redis
from openai import OpenAI
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv()

# 实例化 DeepSeek API 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'), 
    base_url=os.getenv('DEEPSEEK_BASE_URL')
)


def hash_question(question):
    """使用 MD5 对问题进行哈希，并添加前缀"""
    # 计算 MD5 哈希
    md5_hash = hashlib.md5(question.encode('utf-8')).hexdigest()
    # 添加前缀
    return f"{'prefix:'}{md5_hash}"

def call_deepseek_chat(question):
    """调用 deepseek-chat 模型"""
    # 生成哈希键
    hashed_question = hash_question(question)
    
    # 调用模型
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": question}]
    )
    
    # 获取模型的回答
    answer = response.choices[0].message.content
    
    return hashed_question, answer  

&emsp;&emsp;进行模型调用，得到单次请求下哈希键和模型响应。代码如下：

In [9]:
question = "你好，我是木羽，很高兴认识你"
hashed_question, answer = call_deepseek_chat(question)
print(f"哈希键: {hashed_question}")
print(f"模型响应: {answer}")

哈希键: prefix:f6c96a1c978407d928bd19e7f13ee18f
模型响应: 你好，木羽！很高兴认识你！😊 今天有什么我可以帮你的吗？或者你想聊聊什么话题呢？


- 检查Redis是否缓存响应

&emsp;&emsp;这种情况下，因为还没有把上述的哈希键和模型响应存储到`Redis`中，所以缓存中不存在对应的响应。代码如下：

In [12]:
cached_response = r.get(hashed_question)
if cached_response:
    print(f"从缓存中获取响应: {cached_response.decode('utf-8')}")
else:
    print("缓存中不存在对应的响应")

缓存中不存在对应的响应


- 添加哈希键和模型响应到Redis

In [13]:
r.set(hashed_question, answer)

True

- 再次查询Redis中是否存在对应的缓存响应

In [14]:
cached_response = r.get(hashed_question)
if cached_response:
    print(f"从缓存中获取响应: {cached_response.decode('utf-8')}")
else:
    print("缓存中不存在对应的响应")

从缓存中获取响应: 你好，木羽！很高兴认识你！😊 今天有什么我可以帮你的吗？或者你想聊聊什么话题呢？


&emsp;&emsp;可以看到，现在缓存中已经存在对应的响应了。这样当下次提交相同的提示时，`Redis`将返回缓存的响应，而不是查询`API`，从而降低`API`延迟和成本。


&emsp;&emsp;因此，在了解了分步处理的过程后，我们接下来将完整的流程整合到一起，并进行测试。代码如下：

- 使用哈希键在`Redis`中查找是否存在对应的缓存响应

In [16]:
import hashlib
import redis
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

# 加载 .env 文件
load_dotenv()

# 实例化 DeepSeek API 客户端
client = OpenAI(
    api_key=os.getenv('DEEPSEEK_API_KEY'), 
    base_url=os.getenv('DEEPSEEK_BASE_URL')
)

# 连接到 Redis
r = redis.Redis(
    host='192.168.110.131',   # 这里的`host`和`port`需要根据实际情况进行修改
    password='snowball2019',
    port=6379, 
    db=0) 

# 添加前缀
PREFIX = "deepseek:"

def hash_question(question):
    """使用 MD5 对问题进行哈希，并添加前缀"""
    md5_hash = hashlib.md5(question.encode('utf-8')).hexdigest()
    return f"{PREFIX}{md5_hash}"

def call_deepseek_chat(question):
    """调用 deepseek-chat 模型"""
    # 生成哈希键
    hashed_key = hash_question(question)
    
    # 开始计时
    start_time = time.time()
    
    # 检查缓存
    cached_response = r.get(hashed_key)
    if cached_response:
        # 解码缓存响应
        cached_response_decoded = cached_response.decode('utf-8')
        elapsed_time = time.time() - start_time  # 计算命中缓存的时间
        print(f"命中缓存: {cached_response_decoded}")
        print(f"处理时间（命中缓存）: {elapsed_time:.4f} 秒")
        return cached_response_decoded
    
    # 如果缓存不存在，调用模型
    model_start_time = time.time()  # 记录调用模型的开始时间
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": question}]
    )
    
    # 获取模型的回答
    answer = response.choices[0].message.content
    
    # 将结果存入缓存
    r.set(hashed_key, answer)
    
    model_elapsed_time = time.time() - model_start_time  # 计算调用模型的时间
    total_elapsed_time = time.time() - start_time  # 计算总时间
    print(f"调用模型并缓存响应: {answer}")
    print(f"处理时间（未命中缓存）: {model_elapsed_time:.4f} 秒")
    print(f"总处理时间: {total_elapsed_time:.4f} 秒")
    
    return answer


# 测试调用
if __name__ == "__main__":
    question = "什么是大模型？"
    response = call_deepseek_chat(question)
    # print(f"模型响应: {response}")

命中缓存: 大模型（Large Model）通常指的是具有大量参数的机器学习模型，尤其是在自然语言处理（NLP）领域中的大规模预训练语言模型。这些模型通过在大规模数据集上进行预训练，学习到丰富的语言表示，从而在各种下游任务中表现出色。

### 大模型的特点：
1. **参数量大**：大模型的参数量通常在数亿到数千亿之间。例如，OpenAI的GPT-3模型有1750亿个参数。
2. **预训练与微调**：大模型通常在大规模文本数据上进行预训练，学习通用的语言表示。然后，可以通过微调（Fine-tuning）来适应特定的任务，如文本分类、机器翻译、问答系统等。
3. **多任务能力**：由于大模型在预训练过程中学习了广泛的语言知识，它们通常能够处理多种任务，而不需要为每个任务单独设计模型。
4. **计算资源需求高**：训练和部署大模型需要大量的计算资源，包括高性能的GPU或TPU集群，以及大量的存储和内存。

### 常见的大模型：
- **GPT系列**：由OpenAI开发，包括GPT-2、GPT-3等，广泛应用于文本生成、对话系统等任务。
- **BERT**：由Google开发，采用双向Transformer架构，适用于文本分类、问答等任务。
- **T5**：由Google开发，将各种NLP任务统一为文本到文本的转换任务。
- **PaLM**：由Google开发，具有5400亿参数，是目前最大的语言模型之一。

### 应用场景：
- **自然语言理解**：如文本分类、情感分析、命名实体识别等。
- **自然语言生成**：如文本生成、对话系统、机器翻译等。
- **问答系统**：如智能客服、知识问答等。
- **代码生成**：如GitHub Copilot等工具，利用大模型生成代码。

### 挑战：
- **计算成本**：训练和部署大模型需要大量的计算资源，成本高昂。
- **数据需求**：大模型需要大规模的高质量数据进行预训练。
- **模型解释性**：大模型的决策过程往往难以解释，存在“黑箱”问题。
- **伦理与安全**：大模型可能生成有害内容或存在偏见，需要谨慎处理。

总的来说，大模型在推动人工智能技术发展的同时，也带来了新的挑战和机遇。
处理时间（命中缓存）: 0.0020 秒


&emsp;&emsp;通过上面的示例，当用户提交相同的提示（问题）时，`Redis`将返回缓存的响应，而不是去请求大模型服务的`API`，从而使响应时间大大缩短。这就是借助`Redis`的缓存机制可以达到的效果。


&emsp;&emsp;但是目前实现的这个版本存在一个问题：<font color='red'>我们通过`MD5`对`Prompt`进行哈希处理，同时也是根据这个哈希值作为缓存键进行存储和查询，所以意味着一旦`Prompt`发生变化，其哈希值就会发生变化，从而导致缓存不到结构。</font>比如:"什么是大模型？"、"大模型是什么？"，这两个`Prompt`的哈希值是不同的，但其实意思是完全一样的。

&emsp;&emsp;因此，更加常用的一种方法是在匹配过程中融入动态的语义相似度匹配。


## 2.3 基于语义相似度匹配的的 Redis 缓存实现

&emsp;&emsp;正如我们提出的这个问题：“什么是大模型？”、“大模型是什么？”、“如何理解大模型？”，这三个`Prompt`的语义是基本一致的，完全可以用相同的响应来进行回复。那么如何在实际的请求中将这种类似的问题进行匹配，从而达到命中缓存的效果呢？

&emsp;&emsp;解决的方法很容易想到：<font color='red'>不同文本间的语义相似度可以借助`Embedding`来实现</font>。通过将文本转化成向量，然后计算向量之间的余弦相似度，从而判断两个文本是否相似。如果相似，则认为它们是同一个问题，就可以直接提取已缓存问题的响应，尽管它们的`Prompt`不同。（哈希值也不同）

&emsp;&emsp;如果大家没有`Embedding`的基础，可以先行学习`llm_backend\docs\supplementary_course\Ch 17.1 Embedding技术快速入门.ipynb`，对应的视频链接地址：[【赠送】2024大模型技术实战-Part 7. OpenAI Embedding 模型实战](https://appze9inzwc2314.pc.xiaoe-tech.com/p/t_pc/course_pc_detail/video/v_66f2a1b0e4b023c05882a8de?product_id=course_2senjt9B0v1Lprx8OseacXz13rz)，这里我们不作为重点展开讲解。

&emsp;&emsp;`Embedding`模型像对话模型一样，同样可以使用在线`API`版本或者借助`Ollama`框架启动本地版本，并通过`REST API`来进行使用。这里我们就使用`Ollama`启动的本地`Embedding`来给大家进一步讲解实际的实现流程。

### 2.3.1 使用 Ollama 接入 Embedding 模型

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251208767.png" width=80%></div>

&emsp;&emsp;`Ollama` 除了可以接入对话、推理类模型外，还可以接入 `Embedding` 模型。并且部署和使用方法基本与对话、推理类模型保持一致。官方下载地址：https://ollama.com/search?c=embedding ，可点击链接进行查看：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251006679.png" width=80%></div>

&emsp;&emsp;关于如何选择`Embedding`模型，这个问题本质上不存在哪个`Embedding`最好的说法，同时也并没有比较通用且大家都认可的评测数据、流程等，往往还是需要结合自己的实际数据情况加上构建流程评测出来的效果，来进行综合评估。所以这里给大家推荐一个相对完善的`Embedidng`模型评估开源项目，同时也是一个`RAG`的解决方案：https://github.com/timescale/pgai?ref=timescale.ghost.io

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502261900608.png" width=80%></div>

&emsp;&emsp;该开源项目针对`Ollama`支持的`Embedding`模型做了一些基础的评测，如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>常规参数</font></p>
<div class="center">

| 模型名称               | 优势                                         | 劣势                                         |
|----------------------|--------------------------------------------|--------------------------------------------|
| **bge-m3**           | <font color="red">整体检索准确率最高</font>                         | 在不清楚和含糊不清的问题上表现较差，最低准确率 |
|                      | 在长问题上表现出色                         |                                            |
| **mxbai-embed-large**| 尺寸小               | 在简短和直接的问题上表现不如其他模型      |
|                      | 在上下文较重的问题上表现良好               |                                            |
|                      | 长问题表现良好                             |                                            |
| **nomic-embed-text** | 在简短和直接的问题上表现优异               | 整体表现排名最后                          |
|                      | 在长问题上也取得了较好性能                 |                                            |
</div>

&emsp;&emsp;这里我们选择`bge-m3`模型进行下载使用。点击具体的模型，可以查看模型的详细信息，如模型的大小、支持的设备、支持的参数等。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251046299.png" width=80%></div>

&emsp;&emsp;首先需要确定服务器上的`Ollama`服务处于运行状态，通过如下命令查看：

```bash
    systemctl status ollama.service
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251046128.png" width=100%></div>

&emsp;&emsp;如果服务未处于运行状态，则可以通过如下命令启动：

```bash
    systemctl start ollama.service
```



&emsp;&emsp;启动`Ollama`服务后，通过如下命令下载并启动`bge-m3`模型：

```bash
    ollama run bge-m3
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251100592.png" width=100%></div>

&emsp;&emsp;需要说明的是，最后一行`Error: "bge-m3" does not support generation`，表示`bge-m3`模型不支持在类似对话模型启动后可以在命令行进行问答，需要通过`Ollama`的 `REST API` 进行使用。


### 2.3.2 Ollama Embedding REST API 使用

&emsp;&emsp;`Ollama` 提供给`Embedding` 模型使用的 `REST API` 接口为：`/api/embed`。其可以支持单个输入和多个输入的请求。具体可用的参数如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>常规参数</font></p>
<div class="center">

| 参数名       | 类型               | 描述                                                         |
|------------|------------------|------------------------------------------------------------|
| <font color="red">**model**</font>      | 字符串            | 用于生成嵌入的模型名称                                       |
| <font color="red">**input**</font>      | 字符串或字符串列表 | 要生成嵌入的文本或文本列表                                   |
| truncate    | 布尔值            | 是否截断每个输入的末尾以适应上下文长度。若为 false 且超出上下文长度则返回错误。默认值为 true |
| options    | 对象              | 额外的模型参数，例如温度等   |
| keep_alive | 字符串            | 控制模型在请求后保持加载到内存中的时间（默认：5分钟）       |
</div>

&emsp;&emsp;同样，只要了解了某个服务的`REST API` 接口以及其请求参数，就都可以通过`Python` 的`requests` 库进行调用。代码如下：

In [17]:
import requests
import json

# 定义 API 端点
url = "http://192.168.110.131:11434/api/embed"    # 这里需要根据实际情况进行修改

# 单个输入的请求示例
single_input_payload = {
    "model": "bge-m3",   # 这里替换成具体的模型名称
    "input": "你好，我是木羽，很高兴认识你"   # 这里替换成具体的输入文本    
}

# 发送 POST 请求
response_single = requests.post(url, json=single_input_payload)

# 检查响应
if response_single.status_code == 200:
    print("Single Input Response:")
    response_data_single = response_single.json()
    print(json.dumps(response_data_single, indent=2))
else:
    print(f"Error: {response_single.status_code} - {response_single.text}")


Single Input Response:
{
  "model": "bge-m3",
  "embeddings": [
    [
      -0.01922286,
      -0.041361455,
      -0.012615179,
      -0.038346406,
      -0.006061248,
      -0.056141857,
      0.024635639,
      -0.048374478,
      0.021608315,
      0.028389402,
      0.011415558,
      -0.008346135,
      -0.01575168,
      0.011952146,
      0.0151369395,
      -0.0076897917,
      0.019403733,
      -0.03635689,
      0.01607708,
      -0.030296508,
      -0.018613543,
      -0.0064844512,
      -0.00029064808,
      -0.010478618,
      0.029013928,
      0.01946554,
      -0.055293787,
      -0.0024098635,
      0.025135659,
      -0.04489269,
      0.027139282,
      0.015564225,
      0.014931622,
      -0.04502287,
      -0.023891093,
      -0.024381446,
      0.033852994,
      -0.0074271495,
      -0.035470333,
      -0.015253574,
      0.038780138,
      -0.010637472,
      0.050409704,
      -0.047934886,
      -0.0071231266,
      -0.02649895,
      -0.049684074,
      0

&emsp;&emsp;可以提取出`response_single.json()` 中的 `embeddings` 字段，这个字段是一个列表，里面包含了若干个浮点数。而这些浮点数，就是`bge-m3` 模型的嵌入结果，用来表示输入文本`"你好，我是木羽，很高兴认识你"`这句话的语义。

In [ ]:
# 提取嵌入结果
response_data_single["embeddings"]

[[-0.01922286,
  -0.041361455,
  -0.012615179,
  -0.038346406,
  -0.006061248,
  -0.056141857,
  0.024635639,
  -0.048374478,
  0.021608315,
  0.028389402,
  0.011415558,
  -0.008346135,
  -0.01575168,
  0.011952146,
  0.0151369395,
  -0.0076897917,
  0.019403733,
  -0.03635689,
  0.01607708,
  -0.030296508,
  -0.018613543,
  -0.0064844512,
  -0.00029064808,
  -0.010478618,
  0.029013928,
  0.01946554,
  -0.055293787,
  -0.0024098635,
  0.025135659,
  -0.04489269,
  0.027139282,
  0.015564225,
  0.014931622,
  -0.04502287,
  -0.023891093,
  -0.024381446,
  0.033852994,
  -0.0074271495,
  -0.035470333,
  -0.015253574,
  0.038780138,
  -0.010637472,
  0.050409704,
  -0.047934886,
  -0.0071231266,
  -0.02649895,
  -0.049684074,
  0.01412076,
  0.008318819,
  -0.0024914725,
  -0.014517145,
  -0.006007267,
  0.04962171,
  -0.0011974386,
  -0.045221522,
  0.026687985,
  0.036642727,
  -0.018013492,
  0.0062242304,
  -0.037533417,
  -0.026383227,
  0.0430352,
  0.023990763,
  -0.046562776,
  

&emsp;&emsp;具体用多少个浮点数来表示输入文本的语义，取决于具体的模型。比如我们使用的`bge-m3` 模型，其返回的嵌入结果就是一个包含`1024`个浮点数的列表。

In [ ]:
len(response_data_single["embeddings"][0])

1024

&emsp;&emsp;大家可以在`huggingface` 或者 `modelScope` 等平台上，找到`bge-m3` 模型的详细架构、评测信息及论文。我们这里不作为重点展开讲解。`bge-m3` 模型在`modelScope` 平台上的地址为：https://modelscope.cn/models/BAAI/bge-m3

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202502251128661.png" width=80%></div>

&emsp;&emsp; 所以大家可以简单理解为, 经过`bge-m3` 模型处理后的结果, 就是将输入的文本转换为一个包含`1024`个浮点数的列表，并且用这个列表来表示输入文本的语义。即 response_data_single["embeddings"][0] = "你好，我是木羽，很高兴认识你" 


&emsp;&emsp; 同时，`bge-m3` 模型还支持多个输入的请求，即可以一次性输入多个文本，然后返回每个文本的嵌入结果。代码如下：

In [ ]:
import requests
import json

# 定义 API 端点
url = "http://192.168.110.131:11434/api/embed"    # 这里需要根据实际情况进行修改


# 多个输入的请求示例
multiple_input_payload = {
    "model": "bge-m3",
    "input": ["天为什么是蓝色的？", 
              "草为什么是绿色的？"]
}

# 发送 POST 请求
response_multiple = requests.post(url, json=multiple_input_payload)

# 检查响应
if response_multiple.status_code == 200:
    print("\nMultiple Input Response:")
    response_data_multiple = response_multiple.json()
    print(json.dumps(response_data_multiple, indent=2))
    
    # 提取嵌入结果
    multiple_embeddings = response_data_multiple.get("embeddings", [])
else:
    print(f"Error: {response_multiple.status_code} - {response_multiple.text}")


Multiple Input Response:
{
  "model": "bge-m3",
  "embeddings": [
    [
      -0.042137332,
      0.005296808,
      -0.037715282,
      -0.06215416,
      -0.030467484,
      -0.007836851,
      -0.043893967,
      0.0022080292,
      0.016335009,
      -0.0117954165,
      -0.004529275,
      0.025638063,
      0.007796983,
      0.0063098017,
      0.018519185,
      -0.022322154,
      0.029461581,
      0.022574248,
      0.00038702495,
      0.019533236,
      -0.009881311,
      -0.0383717,
      -0.032535516,
      0.016021805,
      -0.006125824,
      0.0075718816,
      0.016597407,
      -0.017723264,
      -0.0002764759,
      0.001488903,
      0.004165976,
      0.0616205,
      -0.029179199,
      -0.022410607,
      0.0003814827,
      -0.017742423,
      -0.005474743,
      0.0049243644,
      -0.040230345,
      -0.032896325,
      0.009739351,
      -0.03913332,
      0.017520385,
      -0.00029088696,
      0.027668685,
      -0.046231378,
      -0.011650233,
    

In [ ]:
print("Total embeddings:", len(multiple_embeddings))
print("First embedding:", len(multiple_embeddings[0]), multiple_embeddings[0])  # 表示天为什么是蓝色的？
print("Second embedding:", len(multiple_embeddings[1]), multiple_embeddings[1])  # 表示草为什么是绿色的？

Total embeddings: 2
First embedding: 1024 [-0.042137332, 0.005296808, -0.037715282, -0.06215416, -0.030467484, -0.007836851, -0.043893967, 0.0022080292, 0.016335009, -0.0117954165, -0.004529275, 0.025638063, 0.007796983, 0.0063098017, 0.018519185, -0.022322154, 0.029461581, 0.022574248, 0.00038702495, 0.019533236, -0.009881311, -0.0383717, -0.032535516, 0.016021805, -0.006125824, 0.0075718816, 0.016597407, -0.017723264, -0.0002764759, 0.001488903, 0.004165976, 0.0616205, -0.029179199, -0.022410607, 0.0003814827, -0.017742423, -0.005474743, 0.0049243644, -0.040230345, -0.032896325, 0.009739351, -0.03913332, 0.017520385, -0.00029088696, 0.027668685, -0.046231378, -0.011650233, -0.03293465, -0.04470256, -0.0478838, 0.008582121, 0.0038168635, 0.088327065, -0.016265068, 0.011740456, -0.0029550614, 0.028078863, -0.030943036, -0.043092407, -0.028814875, -0.014810867, -0.0020290688, -0.041463394, 0.008612065, -0.013520107, 0.08999495, 0.02035601, -0.0044486164, -0.020074788, -0.004833883, -0.0

### 2.3.3 Ollama 使用 OpenAI 兼容的 Embedding 模型接口


&emsp;&emsp;`Ollama`启动的`Embedding REST API` 同样是兼容了`OpenAI` 的接口规范，因此也可以用如下方式进行调用：

In [ ]:
from openai import OpenAI
client = OpenAI(base_url="http://192.168.110.131:11434/v1")  # 需要根据自己的实际情况调整

response = client.embeddings.create(
  model="bge-m3",  # 这里替换成 bge-m3 模型
  input="你好，我是木羽，很高兴认识你",
)

print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.01922286, -0.041361455, -0.012615179, -0.038346406, -0.006061248, -0.056141857, 0.024635639, -0.048374478, 0.021608315, 0.028389402, 0.011415558, -0.008346135, -0.01575168, 0.011952146, 0.0151369395, -0.0076897917, 0.019403733, -0.03635689, 0.01607708, -0.030296508, -0.018613543, -0.0064844512, -0.00029064808, -0.010478618, 0.029013928, 0.01946554, -0.055293787, -0.0024098635, 0.025135659, -0.04489269, 0.027139282, 0.015564225, 0.014931622, -0.04502287, -0.023891093, -0.024381446, 0.033852994, -0.0074271495, -0.035470333, -0.015253574, 0.038780138, -0.010637472, 0.050409704, -0.047934886, -0.0071231266, -0.02649895, -0.049684074, 0.01412076, 0.008318819, -0.0024914725, -0.014517145, -0.006007267, 0.04962171, -0.0011974386, -0.045221522, 0.026687985, 0.036642727, -0.018013492, 0.0062242304, -0.037533417, -0.026383227, 0.0430352, 0.023990763, -0.046562776, 0.005052609, 0.1078952, -0.0252689, -0.036592163, -0.017526496, -0.020836545, -

In [ ]:
print(response.data[0].embedding)

[-0.01922286, -0.041361455, -0.012615179, -0.038346406, -0.006061248, -0.056141857, 0.024635639, -0.048374478, 0.021608315, 0.028389402, 0.011415558, -0.008346135, -0.01575168, 0.011952146, 0.0151369395, -0.0076897917, 0.019403733, -0.03635689, 0.01607708, -0.030296508, -0.018613543, -0.0064844512, -0.00029064808, -0.010478618, 0.029013928, 0.01946554, -0.055293787, -0.0024098635, 0.025135659, -0.04489269, 0.027139282, 0.015564225, 0.014931622, -0.04502287, -0.023891093, -0.024381446, 0.033852994, -0.0074271495, -0.035470333, -0.015253574, 0.038780138, -0.010637472, 0.050409704, -0.047934886, -0.0071231266, -0.02649895, -0.049684074, 0.01412076, 0.008318819, -0.0024914725, -0.014517145, -0.006007267, 0.04962171, -0.0011974386, -0.045221522, 0.026687985, 0.036642727, -0.018013492, 0.0062242304, -0.037533417, -0.026383227, 0.0430352, 0.023990763, -0.046562776, 0.005052609, 0.1078952, -0.0252689, -0.036592163, -0.017526496, -0.020836545, -0.014016081, 0.0073044035, 0.018462053, 0.00972798

&emsp;&emsp;`Ollama` 兼容的 `OpenAI` 接口，其返回的结果与 `Ollama` 的 `REST API` 接口返回的结果是一样的。同时`input` 参数也同样支持单个输入和多个输入。如果需要嵌入多个文本，则需要将`input` 参数设置为列表，如下所示：

In [ ]:
from openai import OpenAI
client = OpenAI(base_url="http://192.168.110.131:11434/v1")

batch_input = ["天为什么是蓝色的？", "草为什么是绿色的？"]


response = client.embeddings.create(
  model="bge-m3",
  input=batch_input
)

print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.042137332, 0.005296808, -0.037715282, -0.06215416, -0.030467484, -0.007836851, -0.043893967, 0.0022080292, 0.016335009, -0.0117954165, -0.004529275, 0.025638063, 0.007796983, 0.0063098017, 0.018519185, -0.022322154, 0.029461581, 0.022574248, 0.00038702495, 0.019533236, -0.009881311, -0.0383717, -0.032535516, 0.016021805, -0.006125824, 0.0075718816, 0.016597407, -0.017723264, -0.0002764759, 0.001488903, 0.004165976, 0.0616205, -0.029179199, -0.022410607, 0.0003814827, -0.017742423, -0.005474743, 0.0049243644, -0.040230345, -0.032896325, 0.009739351, -0.03913332, 0.017520385, -0.00029088696, 0.027668685, -0.046231378, -0.011650233, -0.03293465, -0.04470256, -0.0478838, 0.008582121, 0.0038168635, 0.088327065, -0.016265068, 0.011740456, -0.0029550614, 0.028078863, -0.030943036, -0.043092407, -0.028814875, -0.014810867, -0.0020290688, -0.041463394, 0.008612065, -0.013520107, 0.08999495, 0.02035601, -0.0044486164, -0.020074788, -0.0048338

In [ ]:
print(len(response.data))
print(len(response.data[0].embedding))
print(len(response.data[1].embedding))


2
1024
1024


&emsp;&emsp;至此，`Ollama`启动并使用`Embedding`模型的相关使用方法就已经介绍完毕，并没有很复杂的流程，同时其参数也比较容易理解。

### 2.2.4 动态 Redis 缓存检索实现完整流程


- Step 1 ：基础设置和依赖导入

In [19]:
import numpy as np
from typing import List
import redis
import requests
import hashlib

- Step 2：实现基础的 Embedding 功能

&emsp;&emsp;这里我们实现一个文本 `Embedding` 类，通过 `Ollama` 的 `/api/embed` 的 `REST API` 接口，将输入文本转换为向量。代码如下：

In [20]:
class OllamaEmbedding:
    def __init__(self, base_url: str = "http://localhost:11434", model: str = "bge-m3"):
        """初始化 Ollama Embedding
        
        Args:
            base_url: Ollama 服务地址
            model: 使用的模型名称
        """
        self.base_url = base_url.rstrip('/')
        self.model = model
        self.api_url = f"{self.base_url}/api/embed"
    
    def embed_query(self, text: str) -> List[float]:
        """单个文本转换为向量
        
        Args:
            text: 输入文本
            
        Returns:
            List[float]: 向量表示
        """
        payload = {
            "model": self.model,
            "input": text
        }
        
        try:
            response = requests.post(self.api_url, json=payload)
            response.raise_for_status()  # 检查请求是否成功
            result = response.json()
            return result["embeddings"][0]  # 返回第一个（也是唯一的）embedding
        except Exception as e:
            print(f"Error generating embedding: {e}")
            return []
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """多个文本转换为向量
        
        Args:
            texts: 输入文本列表
            
        Returns:
            List[List[float]]: 向量表示列表
        """
        payload = {
            "model": self.model,
            "input": texts
        }
        
        try:
            response = requests.post(self.api_url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result["embeddings"]   # 批量的话这里直接返回列表
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            return []

&emsp;&emsp;接下来进行测试，其中需要重点关注的是：要根据自己的实际情况，修改 `base_url` 和 `model` 参数。

In [21]:
# 初始化 embedding 模型
embedding = OllamaEmbedding(
    base_url="http://192.168.110.131:11434",  # 根据自己的实际情况调整 endpoint 
    model="bge-m3"         # 根据自己的实际情况调整模型名称
)

# 测试单个文本
test_text = "请问如何理解大模型技术？"
single_vector = embedding.embed_query(test_text)
print("\n单个文本测试:")
print(f"输入文本: {test_text}")
print(f"向量维度: {len(single_vector)}")
print(f"向量示例: {single_vector[:5]}...")  # 只显示前5个数

# 测试多个文本
test_texts = [
    "什么是大模型？",
    "什么是深度学习？",
    "计算机是如何模拟人类思维的？",
    "你好，请你详细的介绍一下你自己"
]
vectors = embedding.embed_documents(test_texts)
print("\n多个文本测试:")
print(f"输入文本数量: {len(test_texts)}")
print(f"生成向量数量: {len(vectors)}")
print(f"每个向量维度: {len(vectors[0])}")
print(f"第一个向量示例: {vectors[0][:5]}...")  # 只显示第一个向量的前5个数


单个文本测试:
输入文本: 请问如何理解大模型技术？
向量维度: 1024
向量示例: [-0.041555773, -0.04320696, -0.04493023, -0.010034573, 0.004685277]...

多个文本测试:
输入文本数量: 4
生成向量数量: 4
每个向量维度: 1024
第一个向量示例: [-0.029385682, -0.038512684, -0.048128907, -0.019845983, 0.0013321947]...


- Step 3：实现 Redis 连接和基本操作

In [ ]:
class RedisSemanticCache:
    def __init__(self, embedding_model, prefix: str = "cache:", 
                 host='192.168.110.131', password=None,port=6379, db=0):
        """初始化 Redis 缓存
        
        Args:
            embedding_model: 嵌入模型
            prefix: 键前缀
            host: Redis 主机地址
            port: Redis 端口
            db: Redis 数据库号
        """
        self.redis_client = redis.Redis(
            host=host,
            port=port,
            db=db,
            password=password
        )
        self.embedding = embedding_model
        self.prefix = prefix
        
    def _create_key(self, text: str) -> str:
        """使用 MD5 创建确定性的 key
        
        Args:
            text: 输入文本
            
        Returns:
            str: 格式为 "prefix:md5hash" 的 key
        """
        # 将文本转换为 MD5 哈希
        # 1. hash() 函数可以将任意长度的文本转换为一个固定长度的整数
        # 2. 相同的文本会生成相同的哈希值，不同的文本很大概率会生成不同的哈希值
        # 3. 使用哈希值作为 key 比直接使用原文本更节省空间
        # 4. Redis key 的查找会更快
        # 5. 避免原始文本中的特殊字符可能导致的问题
        # 6. 防止文本过长超出 Redis key 的长度限制
        md5_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
        return f"{self.prefix}{md5_hash}"  # 前缀（prefix）可以用于命名空间隔离
    
    def add_text(self, text: str, metadata: dict = None):
        # 生成embedding
        vector = self.embedding.embed_query(text)
        key = self._create_key(text)
        
        # 存储数据
        data = {
            'text': text,  # 存储原始文本
            'vector': str(vector),  # 存储向量
            'metadata': str(metadata or {})  # 存储元数据
        }

        # hset：用于将多个字段-值对存储到哈希表 key 中
        self.redis_client.hset(key, mapping=data) 
        
    def get_text(self, key: str) -> dict:
        # hgetall：获取哈希表中所有字段-值对
        return self.redis_client.hgetall(key)
    
    def list_all_keys(self):
        """列出所有缓存的 keys"""
        pattern = f"{self.prefix}*"
        return [key.decode() for key in self.redis_client.keys(pattern)]

    def get_by_text(self, text: str):
        """通过原始文本查询"""
        key = self._create_key(text)
        result = self.redis_client.hgetall(key)
        if result:
            return {k.decode(): v.decode() for k, v in result.items()}
        return None

    def get_all_texts(self):
        """获取所有存储的文本数据"""
        all_data = []
        for key in self.list_all_keys():
            data = self.redis_client.hgetall(key)
            if data:
                all_data.append({k.decode(): v.decode() for k, v in data.items()})
        return all_data
    
    def clear_all_cache(self):
        """清除所有缓存数据"""
        pattern = f"{self.prefix}*"
        keys = self.redis_client.keys(pattern)
        if keys:
            self.redis_client.delete(*keys)
            print(f"已清除 {len(keys)} 条缓存记录")
        else:
            print("缓存为空")

&emsp;&emsp;接下来进行`Embedding`服务和`Redis`客户端的实例化，代码如下：

In [ ]:
# 初始化 embedding 模型
embedding = OllamaEmbedding(
    base_url="http://192.168.110.131:11434",  # 根据自己的实际情况调整 endpoint 
    model="bge-m3"         # 根据自己的实际情况调整模型名称
)

# 初始化 Redis 缓存
redis_cache = RedisSemanticCache(
    host='192.168.110.131',  # 根据自己的实际情况调整 host
    password='snowball2019',  # 根据自己的实际情况调整 password
    port=6379,  # 根据自己的实际情况调整 port
    db=0,  # 根据自己的实际情况调整 db
    embedding_model=embedding
)

&emsp;&emsp;进行功能测试：

In [ ]:
# 测试添加文本
test_text = "大模型是什么？"
redis_cache.add_text(test_text, metadata={"source": "test"})

In [ ]:
# 测试其他功能

# 查看所有存储的数据
print("所有缓存的 keys:")
print(redis_cache.list_all_keys())

print("\n通过文本查询:")
result = redis_cache.get_by_text("哈哈哈，我是木羽。")
print(result)

print("\n所有存储的数据:")
all_data = redis_cache.get_all_texts()
for item in all_data:
    print("\n---")
    print(f"文本: {item['text']}")
    print(f"元数据: {item['metadata']}")
    print(f"向量前5个值: {eval(item['vector'])[:5]}")  # 只显示向量的前5个值

所有缓存的 keys:
['cache:1f52a2047042efc2f5f23376e48050f4', 'cache:b2aa1020a4693111b5bc3d98e3a95a29', 'cache:30d24988981bb0308b93673cb522fc9d', 'cache:8b769ef1812291fe5ae93aff8432f4d8', 'cache:e5fd5cfff22fcebf1602f31f3f236fba', 'cache:88c8e19aa77a1eda28d1741149d27a33']

通过文本查询:
{'text': '哈哈哈，我是木羽。', 'metadata': "{'source': 'test'}", 'vector': '[-0.020885943, -0.029287342, -0.0026394506, -0.05785478, -0.0098147215, -0.044601303, 0.027128061, -0.0111369565, 0.02082092, 0.009936759, -0.009791479, -0.015923912, -0.02313447, -0.0010398539, 0.02385056, -0.0073330663, 0.031261012, -0.031160781, 0.0114881545, -0.03397157, -0.03666232, -0.0012751914, 0.0015402157, -0.0116541255, 0.028772883, 0.012716129, -0.044960573, 0.005683381, -0.0007116142, -0.04183886, 0.009767413, 0.021586942, -0.004036578, -0.067255035, -0.040330436, -0.015747456, 0.01748378, -0.021539608, -0.04152812, -0.0008141396, 0.023163542, 0.0058186203, 0.04727955, -0.04772443, 0.006917078, -0.031089082, -0.016629642, 0.0032309873, 0.

- Step 4：实现相似度搜索

In [ ]:
def cosine_similarity(vec1: List[float], vec2: List[float]) -> float:
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

class SemanticCache(RedisSemanticCache):
    def similarity_search(self, query: str, threshold: float = 0.8) -> List[dict]:
        # 获取查询文本的embedding
        query_vector = self.embedding.embed_query(query)
        
        # 获取所有缓存的数据
        results = []
        for key in self.redis_client.keys(f"{self.prefix}*"):
            cached_data = self.get_text(key)
            if not cached_data:
                continue
                
            # 计算相似度
            cached_vector = eval(cached_data[b'vector'])  # 将字符串转回列表
            similarity = cosine_similarity(query_vector, cached_vector)
            
            if similarity >= threshold:
                results.append({
                    'text': cached_data[b'text'].decode(),
                    'similarity': similarity,
                    'metadata': eval(cached_data[b'metadata'].decode())
                })
        
        return sorted(results, key=lambda x: x['similarity'], reverse=True)

&emsp;&emsp;进行语义相似度功能测试。代码如下：

In [ ]:
# 测试
semantic_cache = SemanticCache(
    host='192.168.110.131',  # 根据自己的实际情况调整 host
    password='snowball2019',  # 根据自己的实际情况调整 password
    port=6379,  # 根据自己的实际情况调整 port
    db=0,  # 根据自己的实际情况调整 db
    embedding_model=embedding
)


# 测试相似度搜索
query = "什么是大模型？"
results = semantic_cache.similarity_search(query, threshold=0.95)
print("\n搜索结果:")
for result in results:
    print(f"文本: {result['text']}")
    print(f"相似度: {result['similarity']:.3f}")
    print(f"元数据: {result['metadata']}\n")


搜索结果:
文本: 大模型是什么？
相似度: 0.990
元数据: {'source': 'test'}



In [ ]:
all_data = redis_cache.get_all_texts()
for item in all_data:
    print("\n---")
    print(f"文本: {item['text']}")
    print(f"元数据: {item['metadata']}")
    print(f"向量前5个值: {eval(item['vector'])[:5]}")  # 只显示向量的前5个值


---
文本: 请问什么是深度学习？
元数据: {'source': 'test'}
向量前5个值: [-0.032345884, -0.05518718, -0.036534578, -0.01660456, -0.021685775]

---
文本: 我是木羽。
元数据: {'source': 'test'}
向量前5个值: [-0.025715707, -0.025451344, -0.0002246134, -0.059394475, -0.0028480904]

---
文本: 大模型是什么？
元数据: {'source': 'test'}
向量前5个值: [-0.02350533, -0.035435423, -0.0519047, -0.02227806, 6.358864e-05]

---
文本: 你好，请你详细的介绍一下你自己
元数据: {'source': 'test'}
向量前5个值: [-0.042102277, 8.325933e-05, -0.070192784, -0.033349816, -0.017922495]

---
文本: 计算机是如何模拟人类思维的？
元数据: {'source': 'test'}
向量前5个值: [-0.023597915, -0.022504436, -0.017778937, 0.0015630199, 0.0026093498]

---
文本: 哈哈哈，我是木羽。
元数据: {'source': 'test'}
向量前5个值: [-0.020885943, -0.029287342, -0.0026394506, -0.05785478, -0.0098147215]


&emsp;&emsp;通过上述代码，我们可以看到，当查询文本为`"大模型是什么？"`时，搜索结果为`"请问什么是大模型？"`，并且相似度为`0.95`，所以我们现在实现了对用户输入问题的基于语义的缓存搜索。

&emsp;&emsp;现在回归到实际的业务场景中，`Prompt Cache` 的应用逻辑应该是这样的：

1. 当用户输入问题时，先根据用户输入的问题，去缓存中搜索，如果缓存命中（即相似度大于某个阈值），则直接返回缓存中的答案。

2. 如果缓存没有命中，则根据用户输入的问题，执行大模型服务调用（API）逻辑，并在得到问答的结果后，将结果存入缓存中。

3. 以此类推，当用户再次输入相同的问题时，由于缓存中已经存在了该问题的答案，所以可以直接从缓存中返回答案，从而提高回答的效率。

&emsp;&emsp;现在我们就可以根据上述的逻辑，来进行优化，思路如下：


- 实现 OllamaChat 类 构建大模型服务调用（API）逻辑

In [ ]:
class OllamaChat:
    def __init__(self, 
                 base_url: str = "http://192.168.110.131:11434", 
                 model: str = "deepseek-r1:1.5b",
                 redis_cache = None,
                 embedding = None):
        self.base_url = base_url.rstrip('/')
        self.model = model
        self.chat_url = f"{self.base_url}/api/chat"
        self.redis_cache = redis_cache
        self.embedding = embedding
    
    def get_answer_with_cache(self, question: str, similarity_threshold: float = 0.90) -> dict:
        """获取答案（优先从缓存中获取）
        
        Args:
            question: 用户问题
            similarity_threshold: 相似度阈值，超过这个值就使用缓存的答案
        """
        # 获取问题的向量
        query_vector = self.embedding.embed_query(question)
        
        # 搜索最相似的问题
        max_similarity = 0
        cached_answer = None
        
        # 遍历所有缓存
        for key in self.redis_cache.list_all_keys():
            data = self.redis_cache.redis_client.hgetall(key)
            if not data:
                continue
            
            # 获取缓存的向量并计算相似度
            cached_vector = eval(data[b'vector'].decode())
            similarity = self._calculate_similarity(query_vector, cached_vector)
            
            # 更新最大相似度和对应的答案
            if similarity > max_similarity:
                max_similarity = similarity
                if similarity >= similarity_threshold:
                    metadata = eval(data[b'metadata'].decode())
                    cached_answer = {
                        'answer': metadata['answer'],
                        'source': 'cache',
                        'similarity': similarity,
                        'original_question': data[b'text'].decode()
                    }
        
        # 如果找到足够相似的缓存答案，直接返回
        if cached_answer:
            return cached_answer
        
        # 如果没有找到相似的缓存，调用模型获取答案
        answer = self._get_llm_answer(question)
        
        # 将新问答对存入缓存
        self.redis_cache.add_text(
            text=question,
            metadata={
                'answer': answer,
                'source': 'llm'
            }
        )
        
        return {
            'answer': answer,
            'source': 'llm',
            'similarity': 0,
            'original_question': None
        }
    
    def _calculate_similarity(self, vec1, vec2) -> float:
        """计算两个向量的余弦相似度"""
        vec1 = np.array(vec1)
        vec2 = np.array(vec2)
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    
    def _get_llm_answer(self, question: str) -> str:
        """从 LLM 获取答案"""
        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": question
                }
            ],
            "stream": False,
            "options": {
                "temperature": 1.3,
            }
        }
        
        try:
            response = requests.post(self.chat_url, json=payload)
            response.raise_for_status()
            result = response.json()
            return result['message']['content']
        except Exception as e:
            print(f"Error getting answer: {e}")
            return ""

&emsp;&emsp;实例化 `Embedding` 模型和 `Redis` 客户端实例。代码如下：

In [ ]:
# 初始化 embedding 模型
embedding = OllamaEmbedding(
    base_url="http://192.168.110.131:11434",  # 根据自己的实际情况调整 endpoint 
    model="bge-m3"         # 根据自己的实际情况调整模型名称
)

# 初始化 Redis 缓存
redis_cache = RedisSemanticCache(
    host='192.168.110.131',  # 根据自己的实际情况调整 host
    password='snowball2019',  # 根据自己的实际情况调整 password
    port=6379,  # 根据自己的实际情况调整 port
    db=0,  # 根据自己的实际情况调整 db
    embedding_model=embedding
)

&emsp;&emsp;先清空一下`Redis`的缓存：

In [ ]:
# 使用示例
redis_cache.clear_all_cache()

已清除 6 条缓存记录


In [ ]:
print("\n所有存储的数据:")
all_data = redis_cache.get_all_texts()
for item in all_data:
    print("\n---")
    print(f"文本: {item['text']}")
    print(f"元数据: {item['metadata']}")
    print(f"向量前5个值: {eval(item['vector'])[:5]}")  # 只显示向量的前5个值


所有存储的数据:


&emsp;&emsp;这里的返回结果就变成了空，说明现在`Redis`中是没有任何数据的。接下来我们实例化对话模型进行测试。代码如下：

In [ ]:
# 初始化聊天模型
chat_model = OllamaChat(
    redis_cache=redis_cache,
    embedding=embedding
)

# 测试问答
question = "请介绍一下人工智能哈"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"来源: {result['source']}")
print(f"答案: {result['answer']}")

问题: 请介绍一下人工智能哈
来源: llm
答案: <think>
嗯，用户问我关于“人工智能哈”是什么。首先，“人工智能”（Artificial Intelligence, AI）这个概念大家应该很熟悉吧？不过具体叫“AI Hashboard”或者“人工智能哈”听起来有些陌生。

我需要弄清楚这句话可能是指什么。也许是在某个特定平台或项目中的名称。我查了一下，发现这是一个Python库，名为alpinehashboard，用于开发和测试AI模型。这名字里包含了人工智能，所以用户可能是想知道这个工具的作用或者使用方法。

接下来，我要解释一下这个库的主要功能。它的主要用途是帮助开发者快速搭建和运行AI模型的环境。它提供了一个可视化的界面，方便不同层次的开发者操作。比如，AI开发工程师可以在IDE中直接编写代码，并用AI算法进行训练和测试，这样节省了时间并提高效率。

另外，用户可能还关心如何安装和使用这个库，我应该提醒他们如何从官方文档或者示例代码入手。如果他们在本地环境中运行，可以使用pip的命令；如果有网络可用的话，则可以用本地服务器下载并解压仓库中的包。

最后，我会总结一下AI Hashboard的核心优势：简洁高效、快速迭代、资源便捷。这些因素使其成为开发者和研究人员的首选工具。
</think>

您的问题涉及到“人工智能哈”这个表述，可能是一个特定平台或工具的名称，而非传统的“人工智能”的缩写（如AAI）。以下是对“人工智能哈”这一表述进行了解释：

### 1. **人工智能（Artificial Intelligence, AI）**
“人工智能”通常指的是一种由软件实现的、能够模拟人 intelligence 的高级系统。AI 系统能够理解、学习、执行并执行各种行动，例如聊天、翻译、推理、识别图像、规划路径等。

### 2. **人工智能哈**
“人工智能哈”可能是某种特定工具或平台（如“人工智能哈”是一个简称为 “hashboard”的平台，用于开发和训练 AI 模型）的名称。如果你是在寻找具体的代码实现或其他技术信息，请提供更多信息以便进一步解答。

例如：
- 如果是指一个 Python 库用于快速搭建和运行 AI 模型的工具，“alpinehashboard”就是这样的库。
- 如果是某个项目或平台，它专注于展示人工智能应用的实际案例

In [ ]:
# 初始化聊天模型
chat_model = OllamaChat(
    redis_cache=redis_cache,
    embedding=embedding
)

# 测试问答
question = "请介绍一下人工智能"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"来源: {result['source']}")
print(f"答案: {result['answer']}")

问题: 请介绍一下人工智能
来源: cache
答案: <think>
嗯，用户问我关于“人工智能哈”是什么。首先，“人工智能”（Artificial Intelligence, AI）这个概念大家应该很熟悉吧？不过具体叫“AI Hashboard”或者“人工智能哈”听起来有些陌生。

我需要弄清楚这句话可能是指什么。也许是在某个特定平台或项目中的名称。我查了一下，发现这是一个Python库，名为alpinehashboard，用于开发和测试AI模型。这名字里包含了人工智能，所以用户可能是想知道这个工具的作用或者使用方法。

接下来，我要解释一下这个库的主要功能。它的主要用途是帮助开发者快速搭建和运行AI模型的环境。它提供了一个可视化的界面，方便不同层次的开发者操作。比如，AI开发工程师可以在IDE中直接编写代码，并用AI算法进行训练和测试，这样节省了时间并提高效率。

另外，用户可能还关心如何安装和使用这个库，我应该提醒他们如何从官方文档或者示例代码入手。如果他们在本地环境中运行，可以使用pip的命令；如果有网络可用的话，则可以用本地服务器下载并解压仓库中的包。

最后，我会总结一下AI Hashboard的核心优势：简洁高效、快速迭代、资源便捷。这些因素使其成为开发者和研究人员的首选工具。
</think>

您的问题涉及到“人工智能哈”这个表述，可能是一个特定平台或工具的名称，而非传统的“人工智能”的缩写（如AAI）。以下是对“人工智能哈”这一表述进行了解释：

### 1. **人工智能（Artificial Intelligence, AI）**
“人工智能”通常指的是一种由软件实现的、能够模拟人 intelligence 的高级系统。AI 系统能够理解、学习、执行并执行各种行动，例如聊天、翻译、推理、识别图像、规划路径等。

### 2. **人工智能哈**
“人工智能哈”可能是某种特定工具或平台（如“人工智能哈”是一个简称为 “hashboard”的平台，用于开发和训练 AI 模型）的名称。如果你是在寻找具体的代码实现或其他技术信息，请提供更多信息以便进一步解答。

例如：
- 如果是指一个 Python 库用于快速搭建和运行 AI 模型的工具，“alpinehashboard”就是这样的库。
- 如果是某个项目或平台，它专注于展示人工智能应用的实际案

In [ ]:
# 初始化聊天模型
chat_model = OllamaChat(
    redis_cache=redis_cache,
    embedding=embedding
)

# 测试问答
question = "如何理解人工智能？"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"来源: {result['source']}")
print(f"答案: {result['answer']}")

问题: 如何理解人工智能？
来源: llm
答案: <think>

</think>

人工智能（AI）指的是能够通过学习和合成人类智能的系统。它通过复杂的算法和模型，如机器学习、深度学习等，从数据中自动提取规律，并做出决策或预测。人工智能在多个领域得到了广泛应用，如自然语言处理、计算机视觉、推荐系统等领域，显著提高了生产力和社会效率。


In [ ]:
# 初始化聊天模型
chat_model = OllamaChat(
    redis_cache=redis_cache,
    embedding=embedding
)

# 测试问答
question = "什么是人工智能？"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"来源: {result['source']}")
print(f"答案: {result['answer']}")

问题: 什么是人工智能？
来源: cache
答案: <think>

</think>

人工智能（AI）指的是能够通过学习和合成人类智能的系统。它通过复杂的算法和模型，如机器学习、深度学习等，从数据中自动提取规律，并做出决策或预测。人工智能在多个领域得到了广泛应用，如自然语言处理、计算机视觉、推荐系统等领域，显著提高了生产力和社会效率。


In [ ]:

# 测试问答
question = "哈哈哈"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"答案: {result['answer']}")
print(f"来源: {result['source']}")

问题: 哈哈哈
答案: <think>

</think>

哈哈，谢谢你的逗笑！ ^w^ 有什么想聊的吗？
来源: llm


In [ ]:

# 测试问答
question = "哈哈哈"
result = chat_model.get_answer_with_cache(question)
print(f"问题: {question}")
print(f"答案: {result['answer']}")
print(f"来源: {result['source']}")

问题: 哈哈哈
答案: <think>

</think>

哈哈，谢谢你的逗笑！ ^w^ 有什么想聊的吗？
来源: cache


In [ ]:
print("\n所有存储的数据:")
all_data = redis_cache.get_all_texts()
for item in all_data:
    print("\n---")
    print(f"文本: {item['text']}")
    print(f"元数据: {item['metadata']}")
    print(f"向量前5个值: {eval(item['vector'])[:5]}")  # 只显示向量的前5个值


所有存储的数据:

---
文本: 哈哈哈
元数据: {'answer': '<think>\n\n</think>\n\n哈哈，谢谢你的逗笑！ ^w^ 有什么想聊的吗？', 'source': 'llm'}
向量前5个值: [-0.016039409, 0.016994113, -0.024663014, -0.0070588915, -0.015972273]

---
文本: 请介绍一下人工智能哈
元数据: {'answer': '<think>\n嗯，用户问我关于“人工智能哈”是什么。首先，“人工智能”（Artificial Intelligence, AI）这个概念大家应该很熟悉吧？不过具体叫“AI Hashboard”或者“人工智能哈”听起来有些陌生。\n\n我需要弄清楚这句话可能是指什么。也许是在某个特定平台或项目中的名称。我查了一下，发现这是一个Python库，名为alpinehashboard，用于开发和测试AI模型。这名字里包含了人工智能，所以用户可能是想知道这个工具的作用或者使用方法。\n\n接下来，我要解释一下这个库的主要功能。它的主要用途是帮助开发者快速搭建和运行AI模型的环境。它提供了一个可视化的界面，方便不同层次的开发者操作。比如，AI开发工程师可以在IDE中直接编写代码，并用AI算法进行训练和测试，这样节省了时间并提高效率。\n\n另外，用户可能还关心如何安装和使用这个库，我应该提醒他们如何从官方文档或者示例代码入手。如果他们在本地环境中运行，可以使用pip的命令；如果有网络可用的话，则可以用本地服务器下载并解压仓库中的包。\n\n最后，我会总结一下AI Hashboard的核心优势：简洁高效、快速迭代、资源便捷。这些因素使其成为开发者和研究人员的首选工具。\n</think>\n\n您的问题涉及到“人工智能哈”这个表述，可能是一个特定平台或工具的名称，而非传统的“人工智能”的缩写（如AAI）。以下是对“人工智能哈”这一表述进行了解释：\n\n### 1. **人工智能（Artificial Intelligence, AI）**\n“人工智能”通常指的是一种由软件实现的、能够模拟人 intelligence 的高级系统。AI 系统能够理解、学习、执行并执行各种行动，例如聊天、翻译、推理、识别图像、规划路径等。\n\n### 2. 

&emsp;&emsp;现在，我们就完成了基于语义的`Redis`的动态缓存检索的完整流程。

&emsp;&emsp;回归到`AssistGen`的项目场景中，则需要考虑因素要更多一些，这主要包括：

1. 添加用户ID到缓存前缀,实现用户隔离;
2. 添加元数据存储(访问时间、访问次数等);
3. 实现自动清理机制;
4. 定期检查缓存大小,当超过最大限制时,删除最久未访问的数据

&emsp;&emsp;`Redis` 缓存功能服务接口核心文件位于：`/llm_backend/app/services/redis_semantic_cache.py`文件。